In [1]:
import numpy as np
from collections import Counter
import string
import re


def distinct_words(corpus):
    """ Determine a list of distinct words for the corpus.

        Params:
            corpus (list of list of strings): corpus of documents
        Return:
            words (list of strings): list of distinct words across the corpus, sorted (using python 'sorted' function)
            num_words (integer): number of distinct words across the corpus
    """
    words = []
    num_words = -1

    words = [word for sentence in corpus for word in sentence]
    words = sorted(set(words))
    num_words = len(words)

    return words, num_words


def compute_co_occurrence_matrix(corpus, window_size=4):
    """ Compute co-occurrence matrix for the given corpus and window_size (default of 4).

        Note: Each word in a document should be at the center of a window. Words near edges will have a smaller
              number of co-occurring words.

              For example, if we take the document "START All that glitters is not gold END" with window size of 4,
              "All" will co-occur with "START", "that", "glitters", "is", and "not".

        Params:
            corpus (list of list of strings): corpus of documents
            window_size (int): size of context window
        Return:
            M (numpy matrix of shape (number of corpus words, number of corpus words)): 
                Co-occurence matrix of word counts. 
                The ordering of the words in the rows/columns should be the same as the ordering of the words given by the distinct_words function.
            word2Ind (dict): dictionary that maps word to index (i.e. row/column number) for matrix M.
    """
    words, num_words = distinct_words(corpus)
    M = None
    word2Ind = {}

    # import numpy as np
    M = np.zeros((len(words), len(words)))
    words = list(words)
    for center_word in words:
        index = words.index(center_word)
        word2Ind[center_word] = index
        for sentence in corpus:
            for index_of_center_word, word in enumerate(sentence):
                if center_word == word:
                    for i in range(window_size):
                        if index_of_center_word-i-1 >= 0:
                            l_n = sentence[index_of_center_word-i-1]
                            M[index, words.index(l_n)] += 1
                        if index_of_center_word + i+1 < len(sentence):
                            r_n = sentence[index_of_center_word+i+1]
                            M[index, words.index(r_n)] += 1

    return M, word2Ind


# Define toy corpus and get co-occurrence matrix
corpus = ["I am studying Natural Language Processing",
          "Natural Language Processing is a subset of Artificial Intelligence"]


# add START and END to each sentence

corpus = [re.findall(r'\b\w+\b', sentence.lower()) for sentence in corpus]


M, word2Ind = compute_co_occurrence_matrix(corpus, window_size=1)

print("Co-occurrence matrix M:")

print(M)
print("\nWord to index mapping:")

print(word2Ind)


def find_similar_words(word, M, word2Ind, k=5):
    """ Find top k most similar words to a given word.

        Params:
            word (string): word to compare similarity
            M (numpy matrix of shape (number of corpus words, number of corpus words)): co-occurrence matrix of word counts
            word2Ind (dict): dictionary that maps word to index (i.e. row/column number) for matrix M
            k (int): number of top similar words to return
        Return:
            similar_words (list of strings): list of top k most similar words
    """
    similar_words = []

    word_index = word2Ind[word]
    word_vector = M[word_index]
    similarity = []
    for i in range(len(M)):
        similarity.append((np.dot(
            M[i], word_vector) / (np.linalg.norm(M[i]) * np.linalg.norm(word_vector)), i))
    similarity = sorted(similarity, reverse=True)
    for i in range(k):
        similar_words.append(list(word2Ind.keys())[list(
            word2Ind.values()).index(similarity[i][1])])

    return similar_words


# Test the function

#  find similarity between all pairs of words from the toy corpus

for word in word2Ind.keys():
    print(f"Words similar to {word}: {find_similar_words(word, M, word2Ind)}")
    print("\n")

Co-occurrence matrix M:
[[0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 1. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 2. 0. 2. 0. 0.]
 [0. 0. 0. 0. 0. 0. 2. 0. 0. 0. 1. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 1. 2. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]]

Word to index mapping:
{'a': 0, 'am': 1, 'artificial': 2, 'i': 3, 'intelligence': 4, 'is': 5, 'language': 6, 'natural': 7, 'of': 8, 'processing': 9, 'studying': 10, 'subset': 11}
Words similar to a: ['a', 'of', 'processing', 'subset', 'studying']


Words similar to am: ['am', 'natural', 'subset', 'studying', 'processing']


Words similar to artificial: ['artificial', 'subset', 'studying', 'processing', 'of']


Words similar to i: ['i', 'studying', 'subset', 'processing', 'of']


Words similar to intel

: 